# Домашнее задание № 3. Исправление опечаток

In [ ]:
!pip install razdel tqdm

In [ ]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from razdel import sentenize
from razdel import tokenize as razdel_tokenize

In [ ]:
# библиотека для отслеживания прогресса
from tqdm.notebook import tqdm

In [ ]:
!pip install textdistance

In [ ]:
import textdistance

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [ ]:
import zipfile
with zipfile.ZipFile(f'data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [ ]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [ ]:
# Корпус отредактированных текстов для словаря правильных слов
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [ ]:
# Попробуем предсказать ошибку простым заглядыванием в словарь. Если слово не в словаре - оно неправильное.
vocab = Counter(re.findall('\w+', corpus.lower()))

In [ ]:
# Косинусное расстояние по мешку символов

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [ ]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [ ]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def recounter_topn(list_to_process, n): # перерасчитывает список, 
    new_counter = Counter()             # в котором только наиболее вероятные из дубликатов
    for word, distance in list_to_process:
        new_counter[word] = distance      
    recounted = new_counter.most_common(n)
    
    return recounted

def extract_most_possible(lists_of_variants, matches): # заменяет дупликаты в списке matches
    for variants in lists_of_variants[:3]:          # на самый вероятный из них
        # т.к. мы в итоге даём 3 самых вероятных варианта исправления
        # больше 3х пар / наборов разбирать нет смысла
        variants_possibility = Counter()
        for variant in variants:
            word, distance = variant
            variants_possibility[word, distance] = P(word)
            if variant in matches:
                matches.remove(variant)
        most_possible = variants_possibility.most_common(1)[0][0]
        matches.append(most_possible)
                 
def extract_equals(matches):
    list_of_equals = [[]] # список списков эквивалентных пар / наборов
                            # пустой список в списке - "костыль"
    for i in range(len(matches)):
        word, distance = matches[i]
        # в списке most_common они уже отсортированы
        # поэтому можно сравнивать с предыдущим значением
        prev_word, prev_distance = matches[i-1]
        equals = [] # эквивалентные пары / наборы
        
        if distance == prev_distance and matches[i-1] not in list_of_equals[-1]: # новая пара
            equals.append(matches[i-1])
            equals.append(matches[i])
            list_of_equals.append(equals)
            
        if distance == prev_distance and matches[i-1] in list_of_equals[-1]: # добавление в уже созданный набор
            list_of_equals[-1].append(matches[i])
         
    list_of_equals = list_of_equals[1:len(list_of_equals)] # убираем "костыль"
        
    return list_of_equals

def get_closest_match_with_metric(text, lookup, topn, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = '{0:.2f}'.format(metric.normalized_similarity(text, word)) # для красоты
    closest_matches = similarities.most_common(topn)
        
    list_of_equals = extract_equals(closest_matches)
    
    if len(list_of_equals) == 0: # если одинаковых кандидатов нет 
        func_result = closest_matches[:3]
        
    else: # для наборов одинаковых кандидатов
        extract_most_possible(list_of_equals, closest_matches)
            
    func_result = recounter_topn(closest_matches, 3)
    
    return func_result

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]    
    closest = get_closest_match_with_metric(text, lookup, 20, metric=metric)

    return closest

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

Проверяем:

In [ ]:
get_closest_hybrid_match('одекватный', X, vec)

[('адекватный', '0.90'), ('адекватной', '0.80'), ('неадекватный', '0.75')]

In [ ]:
get_closest_hybrid_match('вохд', X, vec)

[('вход', '0.75'), ('входов', '0.50'), ('выходов', '0.43')]

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [ ]:
# (1) Составляется словарь правильных слов - уже готово, vocab

# (2) Словарь удалений: ключ - слово с удалением, а значение - правильное слово

dict_of_deletes = {}
for word in vocab:
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    for del_item in deletes:
        if del_item not in dict_of_deletes:
            dict_of_deletes[del_item] = [word]     
        else:
            dict_of_deletes[del_item].append(word)

In [ ]:
def the_most_possible(candidates, mistake):
    possibilities = Counter()
    for candidate in candidates:
        possibilities[candidate] = P(candidate)
    top1candidate = possibilities.most_common(1)[0]
    
    #print(top1candidate)
    #print(mistake, ' - ', P(mistake))
    if top1candidate[1] < P(mistake): # защита от гиперкоррекции
        the_one = mistake
    else:
        the_one = top1candidate[0]
    return the_one

In [ ]:
# (3) Выбор исправления для слова с опечаткой

def make_correct(mistake):
    if mistake in dict_of_deletes: # проверяем само слово с опечаткой
        if len(dict_of_deletes[mistake]) == 1: # если исправление определяется однозначно - это ответ
            correction = dict_of_deletes[mistake][0]
        elif len(dict_of_deletes[mistake]) > 1: # если вариантов несколько - выбираем самый вероятный
            correction = the_most_possible(dict_of_deletes[mistake],mistake)

    else: # если опечатки в словаре нет, делаем варианты удалений     
        mistake_splits     = [(mistake[:i], mistake[i:])    for i in range(len(mistake) + 1)]
        mistake_deletes    = [L + R[1:]               for L, R in mistake_splits if R]
        variants = []
        for item in mistake_deletes:
            if item in dict_of_deletes:
                variants.extend(dict_of_deletes[item])
        if len(variants) == 0: # если ни слова с опечаткой, ни одного его удаления в словаре нет
            correction = mistake
        else:
            correction = the_most_possible(variants, mistake)

    return correction

In [ ]:
make_correct('вдео')

'видео'

In [ ]:
make_correct('гламурный')
#('гламурным', 3.879518450892764e-07)
# гламурный  -  0.0

'гламурным'

In [ ]:
make_correct('девушки')
#('девушкам', 2.1337351479910206e-06)
# девушка  -  4.0928919656918665e-05
# девушки  --  3.685542528348126e-05

'девушка'

Оценка качества:

In [ ]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
    # чтобы два раза не исправлять одно и тоже слово - закешируем его
    # перед тем как считать исправление проверим нет ли его в кеше
        predicted = cashed.get(pair[1], make_correct(pair[1]))
        cashed[pair[1]] = predicted
        print(pair, ' -- ', predicted)
        total += 1
        
        if predicted == pair[0]:
            correct += 1
             
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

('симпатичнейшее', 'симпатичнейшое')  --  симпатичнейшое
('шпионское', 'шпионское')  --  шпионской
('устройство', 'устройство')  --  устройством
('такой', 'такой')  --  такой
('себе', 'себе')  --  себеж
('гламурный', 'гламурный')  --  гламурным
('фотоаппарат', 'фотоаппарат')  --  фотоаппарат
('девушки', 'девушки')  --  девушка
('бонда', 'бонда')  --  бонда
('миниатюрная', 'миниатюрная')  --  миниатюрная
('модель', 'модель')  --  моделью
('камеры', 'камеры')  --  камеры
('superheadz', 'superheadz')  --  superheadz
('clap', 'clap')  --  clamp
('camera', 'camera')  --  camera
('апофеозом', 'опофеозом')  --  апофеозом
('дня', 'дня')  --  дня
('для', 'для')  --  для
('меня', 'меня')  --  меня
('сегодня', 'сегодня')  --  сегодня
('стала', 'стала')  --  стала
('фраза', 'фраза')  --  фразах
('услышанная', 'услышанная')  --  услышанная
('в', 'в')  --  в
('новостях', 'новостях')  --  новостях
('поясним', 'пояним')  --  эпоним
('эту', 'эту')  --  эту
('мысль', 'мысль')  --  мыслью
('получатся', '

In [ ]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6009004502251125
0.35093167701863354
0.36212243022855173


Результаты не очень хорошие, но стали лучше, когда добавила в the_most_possible защиту от гиперкоррекции...
Может, если как-то прикрутить токенизацию, то можно улучшить ещё...

## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса

Алгоритм Byte-Pair Encoding применяется для токенизации. Он сегментирует предложение на токены, исходя из словаря частотных сочетаний символов.
Это работает так: сначала составляется словарь всех уникальных единичных символов предоставленного текста. Потом алгоритм проходится по обучающей выборке и понимает, какие есть частотные сочетания символов. Такие сочетания добавляются в словарь уже как самостоятельные единицы. Сначала пары, потом сочетания из трех, четырех и т.д.. Алгоритм продолжает пополнять словарь пока не дойдёт до определенной длины сочетаний (задаётся переменной). Таким образом, при разборе предложения самые частотные слова будут выделяться как готовые единицы (потому что их цепочка символов наверняка уже есть в словаре). А какие-то редкие слова окажутся составными из словарных цепочек символов. Это удобно, потому что предложение и слова в нём сегментируются не поэлементно, а более крупными кусочками.